In [1]:
import pandas as pd
import pymongo
import json
import datetime as dt
import calendar

In [2]:
# import all cvs from data folder
earthquake_csv = pd.read_csv("../raw/Earthquake_data.csv")
floods_csv = pd.read_csv("../floods_data.csv")
tornado_csv = pd.read_csv("../tor_output.csv")
hurricane_csv = pd.read_csv("../hurricane_cleaned_df.csv")

/Users/jeremyelam/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
earthquake_df = pd.DataFrame(earthquake_csv)
floods_df = pd.DataFrame(floods_csv)
hurricane_df = pd.DataFrame(hurricane_csv)
tornado_df = pd.DataFrame(tornado_csv)

## EARTHQUAKE DATA CLEANING

In [4]:
earthquake_df["Date"] = earthquake_df.Year.map(str)+'-'+ earthquake_df.Month.map(str) + '-' + earthquake_df.Day.map(str)


In [5]:
earthquake_df.Date = pd.to_datetime(earthquake_df.Date)

In [6]:
earthquake_df = earthquake_df.fillna(0)

In [7]:
earthquake_df = earthquake_df.dropna()

In [8]:
earthquake_df.Hour = earthquake_df.Hour.astype(int)

In [9]:
earthquake_df.Minute = earthquake_df.Minute.astype(int)

In [10]:
earthquake_df.Minute = earthquake_df.Minute.map("{:02}".format)  
earthquake_df.Hour  = earthquake_df.Hour.map("{:02}".format)

In [11]:
earthquake_df["Time"] = earthquake_df.Hour.map(str) + ":" + earthquake_df.Minute

In [12]:
earthquake_df = earthquake_df.rename(columns = {"Latitude": "latitude", "Longitude": "longitude"})

In [13]:
earthquake_df.Time = pd.to_datetime(earthquake_df.Time).dt.time

In [14]:
earthquake_df['Month'] = earthquake_df['Month'].apply(lambda x: calendar.month_abbr[x])

In [15]:
earthquake_df

,Year,Month,Day,Hour,Minute,Location,latitude,longitude,Magnitude,Deaths,Injuries,Damage in Millions,Damage Scale,Number of Houses Destroyed,Scale of houses destroyed,Number of Houses Damaged,Scale of houses damaged,Date,Time
0,2017,May,1,12,31,ALASKA: SKAGWAY; CANADA: BRITISH COLUMBIA,59.852,-136.677,6.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-05-01,12:31:00
1,1988,Nov,25,23,46,"CANADA: QUEBEC: SAGUENAY, QUEBEC CITY",48.117,-71.183,5.8,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1988-11-25,23:46:00
2,2010,Jun,23,17,41,"CANADA: QUEBEC: VAL-DES-BOIS, GRACEFIELD",45.880,-75.480,5.2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2010-06-23,17:41:00
3,2001,Oct,12,05,02,CANADA: QUEEN CHARLOTTE ISLANDS,52.630,-132.200,6.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2001-10-12,05:02:00
4,2012,Oct,28,03,04,CANADA: QUEEN CHARLOTTE ISLANDS,52.788,-132.101,7.7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2012-10-28,03:04:00
5,1970,Jun,24,13,09,"CANADA: QUEEN CHARLOTTE ISLANDS, BRITISH COLOMBIA",51.800,-131.000,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1970-06-24,13:09:00
6,2004,Nov,2,10,02,CANADA: VANCOUVER ISLAND,49.277,-128.772,6.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2004-11-02,10:02:00
7,1982,Jan,11,21,41,"CANADA; MAINE: CARIBOU, HAYNESVILLE, PRESQUE ISLE",46.975,-66.659,4.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1982-01-11,21:41:00
8,1968,Sep,25,10,38,MEXICO-GUATEMALA: S CHIAPAS,15.500,-92.700,5.7,20.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1968-09-25,10:38:00
9,1950,Dec,14,14,15,MEXICO: ACAPULCO,16.480,-98.220,7.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1950-12-14,14:15:00


## HURRICANE DATA CLEANING

In [16]:
hurricane_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57095 entries, 0 to 57094
Data columns (total 12 columns):
Unnamed: 0      57095 non-null int64
index           57095 non-null int64
season          57095 non-null int64
basin           57095 non-null object
subbasin        57095 non-null object
name            57095 non-null object
iso_time        57095 non-null object
nature          57095 non-null object
latitude        57095 non-null float64
longitude       57095 non-null float64
usa_wind        57095 non-null float64
usa_pressure    30937 non-null float64
dtypes: float64(4), int64(3), object(5)
memory usage: 5.2+ MB


In [17]:
hurricane_df.head()

,Unnamed: 0,index,season,basin,subbasin,name,iso_time,nature,latitude,longitude,usa_wind,usa_pressure
0,0,1,2018,EP,MM,XAVIER,2018-11-06 06:00:00 UTC,NR,18.9371,-107.832,38.0,1005.0
1,1,2,2018,EP,CP,OLIVIA,2018-09-14 06:00:00 UTC,NR,18.9405,-164.639,29.0,1008.0
2,2,5,2018,EP,CP,OLIVIA,2018-09-14 12:00:00 UTC,NR,18.8559,-165.699,29.0,1008.0
3,3,11,1966,EP,CP,KATHY,1966-10-20 06:00:00 UTC,TS,44.6143,180.000,55.0,NaN
4,4,12,1992,EP,CP,DAN,1992-10-25 06:00:00 UTC,TS,11.6714,180.229,30.0,NaN


In [18]:
hurricane_df = hurricane_df.rename(columns = {"season": "Year"})

In [19]:
hurricane_df = hurricane_df.drop(columns = ["Unnamed: 0"])

In [20]:
hurricane_df[["Date","Time", "Time_Zone"]] = hurricane_df.iso_time.str.split(" ", expand = True)

In [21]:
hurricane_df[["Year", "Month","Day"]] = hurricane_df.Date.str.split("-", expand = True)

In [22]:
hurricane_df.Month = hurricane_df.Month.astype(int)


In [23]:
hurricane_df['Month'] = hurricane_df['Month'].apply(lambda x: calendar.month_abbr[x])

In [24]:
hurricane_df.Date = pd.to_datetime(hurricane_df.Date)
hurricane_df.Time = pd.to_datetime(hurricane_df.Time).dt.time

## FLOOD DATA CLEANING

In [25]:
floods_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34586 entries, 0 to 34585
Data columns (total 17 columns):
episode_id          34586 non-null int64
state               34586 non-null object
event_type          34586 non-null object
event_begin_time    34586 non-null object
event_end_time      34586 non-null object
event_timezone      34586 non-null object
damage_property     34586 non-null float64
event_latitude      34586 non-null float64
event_longitude     34586 non-null float64
cz_type             34586 non-null object
cz_fips_code        34586 non-null int64
damage_crops        34586 non-null float64
source              34586 non-null object
Start_Date          34586 non-null object
Start_Time          34586 non-null object
End_Date            34586 non-null object
End_Time            34586 non-null object
dtypes: float64(4), int64(2), object(11)
memory usage: 4.5+ MB


In [26]:
floods_df[["Date", "Time"]] = floods_df.event_begin_time.str.split(" ", expand = True)

In [27]:
floods_df[["Year", "Month", "Day"]] = floods_df.Start_Date.str.split("-", expand = True)

In [28]:
floods_df.Month = floods_df.Month.astype(int)

In [29]:
floods_df['Month'] = floods_df['Month'].apply(lambda x: calendar.month_abbr[x])

In [30]:
floods_df.Date = pd.to_datetime(floods_df.Date)
floods_df.Time = pd.to_datetime(floods_df.Time).dt.time

In [31]:
floods_df = floods_df.rename(columns = {"event_latitude": "latitude", "event_longitude": "longitude"})


In [32]:
floods_df = floods_df.drop(columns = ["Start_Date", "End_Time", "End_Date","End_Date","Start_Time"])

In [33]:
floods_df.event_begin_time = pd.to_datetime(floods_df.event_begin_time)
floods_df.event_end_time = pd.to_datetime(floods_df.event_end_time)


In [34]:
floods_df.Year = floods_df.Year.astype(int)

## TORNADO DATA CLEANING

In [35]:
tornado_df.head()

,Date,Time,State,Fscale,Injured,Fatalities,Property_loss,SLAT,SLON,ELAT,ELON,Length,Width
0,1950-01-03,11:00:00,MO,3,3.0,0.0,6.0,38.77,-90.22,38.83,-90.03,15.2855,137.111517
1,1950-01-03,11:55:00,IL,3,3.0,0.0,5.0,39.10,-89.30,39.12,-89.23,5.7924,118.829982
2,1950-01-03,16:00:00,OH,1,1.0,0.0,4.0,40.88,-84.58,0.00,0.00,0.1609,9.140768
3,1950-01-13,05:25:00,AR,3,1.0,1.0,3.0,34.40,-94.37,0.00,0.00,0.9654,15.539305
4,1950-01-25,19:30:00,MO,2,5.0,0.0,5.0,37.60,-90.68,37.63,-90.65,3.7007,274.223035


In [36]:
tornado_df[["Year", "Month", "Day"]] = tornado_df.Date.str.split("-", expand = True)


In [37]:
tornado_df.Month = tornado_df.Month.astype(int)

In [38]:
tornado_df['Month'] = tornado_df['Month'].apply(lambda x: calendar.month_abbr[x])

In [39]:
tornado_df = tornado_df.drop(columns = ["Day"])

In [40]:
tornado_df.Date = pd.to_datetime(tornado_df.Date, errors = 'coerce')
# tornado_df.Time = pd.to_datetime(tornado_df.Time, errors = 'coerce')


In [41]:
tornado_df.Property_loss = tornado_df.Property_loss.replace({'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(int)

In [42]:
tornado_df=tornado_df.dropna()

In [43]:
tornado_df.Year = tornado_df.Year.astype(int)

## Change all column to lower case

In [44]:
floods_df.columns = map(str.lower, floods_df.columns)
earthquake_df.columns = map(str.lower, earthquake_df.columns)
hurricane_df.columns = map(str.lower, hurricane_df.columns)
tornado_df.columns = map(str.lower, tornado_df.columns)

## View all DataFrames

In [45]:
floods_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34586 entries, 0 to 34585
Data columns (total 18 columns):
episode_id          34586 non-null int64
state               34586 non-null object
event_type          34586 non-null object
event_begin_time    34586 non-null datetime64[ns]
event_end_time      34586 non-null datetime64[ns]
event_timezone      34586 non-null object
damage_property     34586 non-null float64
latitude            34586 non-null float64
longitude           34586 non-null float64
cz_type             34586 non-null object
cz_fips_code        34586 non-null int64
damage_crops        34586 non-null float64
source              34586 non-null object
date                34586 non-null datetime64[ns]
time                34586 non-null object
year                34586 non-null int64
month               34586 non-null object
day                 34586 non-null object
dtypes: datetime64[ns](3), float64(4), int64(3), object(8)
memory usage: 4.7+ MB


In [46]:
hurricane_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57095 entries, 0 to 57094
Data columns (total 16 columns):
index           57095 non-null int64
year            57095 non-null object
basin           57095 non-null object
subbasin        57095 non-null object
name            57095 non-null object
iso_time        57095 non-null object
nature          57095 non-null object
latitude        57095 non-null float64
longitude       57095 non-null float64
usa_wind        57095 non-null float64
usa_pressure    30937 non-null float64
date            57095 non-null datetime64[ns]
time            57095 non-null object
time_zone       57095 non-null object
month           57095 non-null object
day             57095 non-null object
dtypes: datetime64[ns](1), float64(4), int64(1), object(10)
memory usage: 7.0+ MB


In [47]:
tornado_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66136 entries, 0 to 66140
Data columns (total 15 columns):
date             66136 non-null datetime64[ns]
time             66136 non-null object
state            66136 non-null object
fscale           66136 non-null object
injured          66136 non-null float64
fatalities       66136 non-null float64
property_loss    66136 non-null int64
slat             66136 non-null float64
slon             66136 non-null float64
elat             66136 non-null float64
elon             66136 non-null float64
length           66136 non-null float64
width            66136 non-null float64
year             66136 non-null int64
month            66136 non-null object
dtypes: datetime64[ns](1), float64(8), int64(2), object(4)
memory usage: 8.1+ MB


## Convert all the dataframes into json format

In [48]:
hurricane_data = json.loads(hurricane_df.T.to_json()).values()
floods_data = json.loads(floods_df.T.to_json()).values()
earthquake_data = json.loads(earthquake_df.T.to_json()).values()
tornado_data = json.loads(tornado_df.T.to_json()).values()

## NoSQL

In [49]:
# connect to mongod local server
conn = 'mongodb://localhost:27017'
# create client 
client = pymongo.MongoClient(conn)
# create database
# db.weather_data.drop()
db = client.weather_data


In [50]:
db.hurricane_data.drop()
db.hurricane_data.insert(hurricane_data)

/Users/jeremyelam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[ObjectId('5e22624b37ce9b0615fdc20e'),
 ObjectId('5e22624b37ce9b0615fdc20f'),
 ObjectId('5e22624b37ce9b0615fdc210'),
 ObjectId('5e22624b37ce9b0615fdc211'),
 ObjectId('5e22624b37ce9b0615fdc212'),
 ObjectId('5e22624b37ce9b0615fdc213'),
 ObjectId('5e22624b37ce9b0615fdc214'),
 ObjectId('5e22624b37ce9b0615fdc215'),
 ObjectId('5e22624b37ce9b0615fdc216'),
 ObjectId('5e22624b37ce9b0615fdc217'),
 ObjectId('5e22624b37ce9b0615fdc218'),
 ObjectId('5e22624b37ce9b0615fdc219'),
 ObjectId('5e22624b37ce9b0615fdc21a'),
 ObjectId('5e22624b37ce9b0615fdc21b'),
 ObjectId('5e22624b37ce9b0615fdc21c'),
 ObjectId('5e22624b37ce9b0615fdc21d'),
 ObjectId('5e22624b37ce9b0615fdc21e'),
 ObjectId('5e22624b37ce9b0615fdc21f'),
 ObjectId('5e22624b37ce9b0615fdc220'),
 ObjectId('5e22624b37ce9b0615fdc221'),
 ObjectId('5e22624b37ce9b0615fdc222'),
 ObjectId('5e22624b37ce9b0615fdc223'),
 ObjectId('5e22624b37ce9b0615fdc224'),
 ObjectId('5e22624b37ce9b0615fdc225'),
 ObjectId('5e22624b37ce9b0615fdc226'),
 ObjectId('5e22624b37ce9b

In [51]:
db.flood_data.drop()
db.flood_data.insert(floods_data)

/Users/jeremyelam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[ObjectId('5e22624c37ce9b0615fea115'),
 ObjectId('5e22624c37ce9b0615fea116'),
 ObjectId('5e22624c37ce9b0615fea117'),
 ObjectId('5e22624c37ce9b0615fea118'),
 ObjectId('5e22624c37ce9b0615fea119'),
 ObjectId('5e22624c37ce9b0615fea11a'),
 ObjectId('5e22624c37ce9b0615fea11b'),
 ObjectId('5e22624c37ce9b0615fea11c'),
 ObjectId('5e22624c37ce9b0615fea11d'),
 ObjectId('5e22624c37ce9b0615fea11e'),
 ObjectId('5e22624c37ce9b0615fea11f'),
 ObjectId('5e22624c37ce9b0615fea120'),
 ObjectId('5e22624c37ce9b0615fea121'),
 ObjectId('5e22624c37ce9b0615fea122'),
 ObjectId('5e22624c37ce9b0615fea123'),
 ObjectId('5e22624c37ce9b0615fea124'),
 ObjectId('5e22624c37ce9b0615fea125'),
 ObjectId('5e22624c37ce9b0615fea126'),
 ObjectId('5e22624c37ce9b0615fea127'),
 ObjectId('5e22624c37ce9b0615fea128'),
 ObjectId('5e22624c37ce9b0615fea129'),
 ObjectId('5e22624c37ce9b0615fea12a'),
 ObjectId('5e22624c37ce9b0615fea12b'),
 ObjectId('5e22624c37ce9b0615fea12c'),
 ObjectId('5e22624c37ce9b0615fea12d'),
 ObjectId('5e22624c37ce9b

In [52]:
db.earthquake_data.drop()
db.earthquake_data.insert(earthquake_data)

/Users/jeremyelam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[ObjectId('5e22624d37ce9b0615ff282f'),
 ObjectId('5e22624d37ce9b0615ff2830'),
 ObjectId('5e22624d37ce9b0615ff2831'),
 ObjectId('5e22624d37ce9b0615ff2832'),
 ObjectId('5e22624d37ce9b0615ff2833'),
 ObjectId('5e22624d37ce9b0615ff2834'),
 ObjectId('5e22624d37ce9b0615ff2835'),
 ObjectId('5e22624d37ce9b0615ff2836'),
 ObjectId('5e22624d37ce9b0615ff2837'),
 ObjectId('5e22624d37ce9b0615ff2838'),
 ObjectId('5e22624d37ce9b0615ff2839'),
 ObjectId('5e22624d37ce9b0615ff283a'),
 ObjectId('5e22624d37ce9b0615ff283b'),
 ObjectId('5e22624d37ce9b0615ff283c'),
 ObjectId('5e22624d37ce9b0615ff283d'),
 ObjectId('5e22624d37ce9b0615ff283e'),
 ObjectId('5e22624d37ce9b0615ff283f'),
 ObjectId('5e22624d37ce9b0615ff2840'),
 ObjectId('5e22624d37ce9b0615ff2841'),
 ObjectId('5e22624d37ce9b0615ff2842'),
 ObjectId('5e22624d37ce9b0615ff2843'),
 ObjectId('5e22624d37ce9b0615ff2844'),
 ObjectId('5e22624d37ce9b0615ff2845'),
 ObjectId('5e22624d37ce9b0615ff2846'),
 ObjectId('5e22624d37ce9b0615ff2847'),
 ObjectId('5e22624d37ce9b

In [ ]:
db.tornado_data.drop()
db.tornado_data.insert(tornado_data)

/Users/jeremyelam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  
